In [59]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [60]:
from langchain_groq import ChatGroq

model = ChatGroq(model = "gemma2-9b-it", groq_api_key =  groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019A7B2C0EB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019A7B2C1E10>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [61]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content= "Hi im jacob, a AI Engineer"),
        AIMessage(content= "Hello jacob, its nice to meet you"),
        HumanMessage(content= "Heys whats my name and what do i do?")
    ]
)

AIMessage(content="You told me your name is Jacob, and that you are an AI Engineer!  😊 \n\nIs there anything specific you'd like to talk about related to your work as an AI Engineer? I'm happy to chat about AI, machine learning, or any other topic you're interested in. \n\n", response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 45, 'total_tokens': 112, 'completion_time': 0.121818182, 'prompt_time': 0.000571447, 'queue_time': 0.015041863, 'total_time': 0.122389629}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4ffd8de-3652-4b6c-8315-35f049446cae-0', usage_metadata={'input_tokens': 45, 'output_tokens': 67, 'total_tokens': 112})

In [62]:
## Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [63]:
store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [64]:
config = {"configurable": {"session_id" : "chat1"}}

In [65]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hi im jacob, a AI Engineer")],
    config= config
)

In [66]:
response.content

"Hello Jacob! It's nice to meet you.  That's fascinating, being an AI engineer yourself!  \n\nWhat kind of AI projects are you working on?  I'm always eager to learn more about what other AI folks are up to.\n"

In [67]:
with_message_history.invoke(
    [HumanMessage(content= "What is my name?")],
    config= config
)

AIMessage(content="Your name is Jacob.  \n\nIt seems like you're checking!  😊  Is there anything else I can help you with?\n", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 85, 'total_tokens': 116, 'completion_time': 0.056363636, 'prompt_time': 0.003258634, 'queue_time': 0.011288085, 'total_time': 0.05962227}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3ed4e4e-ab7f-4688-83f5-21c387c876e7-0', usage_metadata={'input_tokens': 85, 'output_tokens': 31, 'total_tokens': 116})

In [68]:
#change the config , means changing the session id

config1 = {"configurable": {"session_id" : "chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content= "Whats my name?")],
    config = config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [69]:
response = with_message_history.invoke(
    [HumanMessage(content= "hey my name is viston")],
    config = config1
)

response.content

"Hey Viston, it's nice to meet you!  What can I do for you today?  😊 \n"

In [70]:
response = with_message_history.invoke(
    [HumanMessage(content= "Whats my name?")],
    config = config1
)

response.content

'Your name is Viston!  We talked about it just a moment ago. 😊  \n\nIs there anything else I can help you with?\n'

### Prompt Template

In [71]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [72]:
chain.invoke({"messages": [HumanMessage(content="Hi my name is viston")]})

AIMessage(content="Hello Viston! It's nice to meet you. \n\nWhat can I help you with today?  😄 \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 31, 'total_tokens': 59, 'completion_time': 0.050909091, 'prompt_time': 0.000350708, 'queue_time': 0.014121001, 'total_time': 0.051259799}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed8cfb99-bb0d-4b6c-a93a-9b7cb67d7f23-0', usage_metadata={'input_tokens': 31, 'output_tokens': 28, 'total_tokens': 59})

In [73]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [74]:
config = {"configurable": {"session_id" : "chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is viston")],
    config=config
)

response.content

"Hello Viston! It's nice to meet you.\n\nWhat can I do for you today? 😊  \n\n"

In [75]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [76]:
prompt

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='You are a helpful assistant. Answer all questions to the best of your ability in {language}.')), MessagesPlaceholder(variable_name='messages')])

In [77]:
response = chain.invoke(
    {"messages": [HumanMessage(content="Hi my name is viston")], "language": "French"}
)

response.content

"Bonjour Viston ! \n\nEnchanté de te rencontrer. Comment puis-je t'aider aujourd'hui ? \n\n"

In [78]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"
)

In [79]:
config = {"configurable": {"session_id" : "chat4"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi my name is viston")], "language": "Hindi"},
    config=config,
)

response.content

'नमस्ते विस्तोन! 👋  आपका नाम सुनकर अच्छा लगा। मैं आपकी मदद करने के लिए तैयार हूँ। आपके कोई सवाल हैं? 😊 \n'

In [80]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

response.content

'आपका नाम विस्तोन है। 😊  \n'

### Managing the converstaion history

In [81]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

In [82]:
messages = [
    SystemMessage(content = "you're a good assistant"),
    HumanMessage(content= "Hi! im bob"),
    AIMessage(content = "Hi"),
    HumanMessage(content= "i like vanilla ice scream"),
    AIMessage(content = "nice"),
    HumanMessage(content= "whats 2 + 2"),
    AIMessage(content = "4"),
    HumanMessage(content= "thanks"),
    AIMessage(content = "no problem"),
    HumanMessage(content= "having fun ?"),
    AIMessage(content = "yes!"),    
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='i like vanilla ice scream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun ?'),
 AIMessage(content='yes!')]

In [83]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [84]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIs there a name you'd like to go by? 😊  I'm happy to remember it!\n"

In [85]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "What is 2 + 2?"  😊 \n\n\nIs there another problem you\'d like to try?\n'

In [86]:
#lets wrap this in message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "chat5"}}

In [87]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to any personal information about you, including your name.  \n\nDo you want to tell me your name? 😊  \n\n"